In [231]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from datetime import datetime
import bs4
import pandas as pd
import time

driver = webdriver.Chrome()
driver.get('https://www.google.co.th/maps/place/%E0%B8%9E%E0%B8%B4%E0%B8%9E%E0%B8%B4%E0%B8%98%E0%B8%A0%E0%B8%B1%E0%B8%93%E0%B8%91%E0%B9%8C%E0%B8%AA%E0%B8%B1%E0%B8%95%E0%B8%A7%E0%B9%8C%E0%B8%99%E0%B9%89%E0%B8%B3%E0%B8%88%E0%B8%B1%E0%B8%87%E0%B8%AB%E0%B8%A7%E0%B8%B1%E0%B8%94%E0%B8%AB%E0%B8%99%E0%B8%AD%E0%B8%87%E0%B8%84%E0%B8%B2%E0%B8%A2/@17.8125742,102.7575107,14z/data=!4m7!3m6!1s0x31247cce615c9f5b:0x6c079a02d19f64ee!8m2!3d17.8093347!4d102.7462548!9m1!1b1?hl=th')
review_data = pd.DataFrame(columns = ['name', 'score', 'time', 'comment'])

time.sleep(5.0)

data = driver.page_source
soup = bs4.BeautifulSoup(data)
all_review_score = soup.find_all('div',{'class':'fontBodySmall'})
allcomment = int(all_review_score[0].text.split()[0].replace(',', ''))

In [ ]:
loaded = 0
n = 5
count = 0
while loaded < allcomment:
    i = 0
    while i < n:
        html = driver.find_elements(By.CLASS_NAME,'DxyBCb')
        html[0].send_keys(Keys.PAGE_DOWN)
        time.sleep(2.0)
        i += 1

    time.sleep(5.0) 
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    currentload = soup.find_all('div',{'class':'jftiEf fontBodyMedium'})
    
    if loaded == len(currentload):
        count += 1
    
    loaded = len(currentload)
    print(loaded)

    if count == 3:
        break
    
    n *= 2


In [234]:
def loaddata():
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    all_review_score = soup.find_all('div',{'class':'jftiEf fontBodyMedium'})

    list_name = []
    list_score = []
    list_time = []
    list_comment = []

    for x in all_review_score:    
        name = x['aria-label']
        profiles = x.findChildren('div',{'class':'DU9Pgb'})
        pf = list(profiles)[0]
        score = pf.findChildren('span', {'class':'kvMYJc'})[0]
        score = score['aria-label'].split()[0]
        stime = pf.findChildren('span', {'class':'rsqaWe'})[0]
        stime = stime.text    
        comment = x.findChildren('span', {'class':'wiI7pd'})
        comment = comment[0].text

        list_name.append(name)
        list_score.append(score)
        list_time.append(stime)
        list_comment.append(comment)

    df = pd.DataFrame([list_name, list_score, list_time, list_comment])
    df = df.transpose()
    df.columns = ['name', 'score', 'time', 'comment']
    return df

df = loaddata()
    

In [235]:
df

,name,score,time,comment
0,ชัยพร รุจิราวรรณ,4,4 เดือนที่แล้ว,"ข้อดี\n- สถานที่กว้างขวาง,สะอาด\n-สัตว์น้ำถือว..."
1,Torpong Chiamtawongse,4,5 เดือนที่แล้ว,หลากหลายสายพันธุ์ที่บางตัวก็เพิ่งเคยเห็นชื่อนี...
2,J Jety S (Ninyada),5,5 เดือนที่แล้ว,ค่าเข้า 50 บาท สำหรับผู้ใหญ่ เด็กสูงไม่เกิน 13...
3,Taksinee Khongthan,5,7 เดือนที่แล้ว,สวย สะอาด ไปไม่ทันโชว์ให้อาหาร มีปลาจำพวกปลา...
4,kkuaob Kaeo,5,2 เดือนที่แล้ว,ชื่นชมปลาชนิดต่างๆ เด็กๆมีความสุขที่ไดเห็นปลาต...
...,...,...,...,...
1085,ป้อม กระบี่,5,ปีที่แล้ว,
1086,Barry Walters,1,เดือนที่แล้ว,
1087,P Thonubol,5,4 ปีที่แล้ว,
1088,Aniwut Khunwong,4,4 ปีที่แล้ว,


In [237]:
df.to_excel(r'results\google.xlsx')